In [1]:
%matplotlib inline

# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from functools import reduce
import os
import pickle

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import sys
sys.path.insert(0, "D:\\Documents\\THU\\food_recipe_gen")

from recipe_1m_analysis.utils import Vocabulary, FOLDER_PATH, DATA_FILES 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0
EOS_token = 1

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [4]:
DATA_FILES

['allingrs_count.pkl',
 'allwords_count.pkl',
 'recipe1m_test.pkl',
 'recipe1m_vocab_ingrs.pkl',
 'recipe1m_vocab_toks.pkl']

In [5]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[3]),'rb') as f:
    vocab_ingrs=pickle.load(f)
    
with open(os.path.join(FOLDER_PATH,DATA_FILES[4]),'rb') as f:
    vocab_tokens=pickle.load(f)

In [6]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[2]),'rb') as f:
    data=pickle.load(f)

pairs=[]
for recipe in data:
    pairs.append([recipe["ingredients"],recipe["tokenized"]])

print(len(pairs))

18745


In [7]:
length=0
for i,pair in enumerate(pairs):
    length_t = len([item for sublist in pair[1] for item in sublist])
    
    if length_t>length:
        length=length_t

length

539

In [8]:
MAX_LENGTH=300

In [9]:
# pairs[16826]

In [10]:
def filterPair(p):
    length=0
    for ingr in p[0]:
        if ingr not in vocab_ingrs.word2idx:
            return False
        
    for sent in p[1]:
        
        for word in sent:
            # TODO check how steps tokenized ? Put into vocab ???
            if word not in vocab_tokens.word2idx:
                return False
        length+=len(sent)
    
    return length < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

pairs = filterPairs(pairs)
print(len(pairs))

14724


In [11]:
# vocab_ingrs.word2idx['quahogs']

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [12]:
def list2idx(vocab, sentence):
    return [vocab.word2idx[word] for word in sentence]


def tensorFromSentence(vocab, sentence,instructions=False):
    if instructions:
        indexes=[]
        for sent in sentence:
            indexes.extend(list2idx(vocab, sent))
    else:
        indexes = list2idx(vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab_ingrs, pair[0])
    target_tensor = tensorFromSentence(vocab_tokens, pair[1],instructions=True)
    return (input_tensor, target_tensor)

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.



In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




**Simple Decoder**


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).




In [14]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Attention Decoder**

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

In [15]:
class AttnDecoderRNN(DecoderRNN):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__(hidden_size,output_size)
        self.attention = Attention(hidden_size, dropout_p=dropout_p, max_length=max_length)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        
        output, attn_weights = self.attention(embedded,hidden,encoder_outputs)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

class Attention(nn.Module):
    def __init__(self,hidden_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.hidden_size = hidden_size
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

    def forward(self,embedded,hidden,encoder_outputs):
        """key:encoder_outputs
        value:hidden
        query: embedded
        """
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        return output,attn_weights
  

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.


Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [36]:
class Seq2seq(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,learning_rate=0.01, max_length=MAX_LENGTH,teacher_forcing_ratio=0.5,attention=True):
        super().__init__()
        # all params in arg storage object ?
        self.attention=attention
        self.encoder = EncoderRNN(input_size, hidden_size).to(device)
        if attention:
            self.decoder = AttnDecoderRNN(hidden_size, output_size, dropout_p=0.1).to(device)
        else:
            self.decoder = DecoderRNN(hidden_size, output_size).to(device)
        
        self.encoder_optimizer = optim.SGD(self.encoder.parameters(), lr=learning_rate)
        self.decoder_optimizer = optim.SGD(self.decoder.parameters(), lr=learning_rate)
        self.max_length=max_length
        
        # Training param
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.learning_rate = learning_rate
        self.criterion = nn.NLLLoss()
    
    def forward(self,input_tensor, target_tensor):
        encoder_hidden = self.encoder.initHidden()
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden = self.encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)

        decoder_hidden = encoder_hidden

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            if self.attention:
                for di in range(target_length):
                    decoder_output, decoder_hidden, decoder_attention = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs)
                    loss += self.criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing
            else:
                for di in range(target_length):
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden)
                    loss += self.criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

        else:
            # Without teacher forcing: use its own predictions as the next input
            if self.attention:
                for di in range(target_length):
                    decoder_output, decoder_hidden, decoder_attention = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += self.criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break
            else:
                for di in range(target_length):
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += self.criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break
        return loss/ target_length



    def train_iter(self,input_tensor, target_tensor):
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        loss = self.forward(input_tensor,target_tensor)

        loss.backward()

        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return loss.item() 
    
    
    def train_process(self,n_iters, print_every=1000, plot_every=100):
        start = time.time()
        plot_losses = []
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every

        training_pairs = [tensorsFromPair(random.choice(pairs))
                          for i in range(n_iters)]

        for iter in range(1, n_iters + 1):
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = self.train_iter(input_tensor, target_tensor)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

        showPlot(plot_losses)
        
    def evaluate(self,sentence):
        # TODO use forward
        with torch.no_grad():
            input_tensor = tensorFromSentence(vocab_ingrs, sentence)
            input_length = input_tensor.size()[0]
            encoder_hidden = self.encoder.initHidden()

            encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = self.encoder(input_tensor[ei],
                                                         encoder_hidden)
                encoder_outputs[ei] += encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            decoder_hidden = encoder_hidden

            decoded_words = []
            decoder_attentions = torch.zeros(self.max_length, self.max_length)

            for di in range(self.max_length):
                decoder_output, decoder_hidden, decoder_attention = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                decoder_attentions[di] = decoder_attention.data
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    decoded_words.append('<EOS>')
                    break
                else:
                    decoded_words.append(vocab_tokens.idx2word[topi.item()])

                decoder_input = topi.squeeze().detach()

            return decoded_words, decoder_attentions[:di + 1]
        
    def evaluateRandomly(self,n=10):
        for i in range(n):
            pair = random.choice(pairs)
            print('>', pair[0])
            print('=', pair[1])
            output_words, attentions = evaluate(encoder, decoder, pair[0])
            output_sentence = ' '.join(output_words)
            print('<', output_sentence)
            print('')

In [37]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Plotting results
----------------

In [38]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Training and Evaluating
=======================

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(len(vocab_ingrs.idx2word.keys()), hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(vocab_tokens.idx2word.keys()), dropout_p=0.1).to(device)

model = Seq2seq(len(vocab_ingrs.idx2word.keys()),hidden_size,len(vocab_tokens.idx2word.keys()))

model.train_process(5000, print_every=100)

1m 43s (- 84m 46s) (100 2%) 9.9568
4m 3s (- 97m 20s) (200 4%) 9.3421
6m 35s (- 103m 22s) (300 6%) 8.5991
9m 13s (- 106m 3s) (400 8%) 7.9830
11m 42s (- 105m 24s) (500 10%) 7.5441
14m 20s (- 105m 11s) (600 12%) 7.2905
17m 0s (- 104m 30s) (700 14%) 7.0544
19m 26s (- 102m 5s) (800 16%) 6.8965


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

Visualizing Attention
---------------------

In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)